Mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

import libraries

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split # replace old code from sklearn.cross_validation import train_test_split

from keras import backend as K
keras.backend.image_data_format()# replace old code K.set_image_dim_ordering('tf')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation, BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam

Define path and data

In [ ]:
PATH = "/content/gdrive/My Drive/Pokemonk" 
print(PATH)
data_path = PATH + '/dataset'
data_dir_list = os.listdir(data_path)

Load image data and resize to 128x128 size, gray scale, i.e. one channel

In [ ]:
img_rows = 128
img_cols = 128
num_channel = 1

img_data_list = []

for dataset in data_dir_list:
    img_list = os.listdir(data_path + '/' + dataset)
    print('Loaded the images of dataset-' + '{}\n'.format(dataset))
    for img in img_list:
        input_img = cv2.imread(data_path + '/' + dataset + '/' + img)# image is in jpg or png format
        input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize = cv2.resize(input_img, (128, 128))
        img_data_list.append(input_img_resize)

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print(img_data.shape)

show image data shape

In [ ]:
if num_channel == 1:
    if K.image_data_format() == 'th':
        img_data = np.expand_dims(img_data, axis=1)
        print(img_data.shape)
    else:
        img_data = np.expand_dims(img_data, axis=3)# axis was 4(cause error) before change to 3
        print(img_data.shape)

else:
    if K.image_data_format() == 'th':
        img_data = np.rollaxis(img_data, 3, 1)
        print(img_data.shape)
# K was defined in loading libraries for Image_data_format

Label of 5 classes of pokemon 

In [ ]:
num_classes = 5

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,), dtype='int64')

labels[0:233] = 0
labels[233:472] = 1
labels[472:711] = 2
labels[711:946] = 3
labels[946:1170] = 4

names = ['bulbasaur', 'charmander', 'mewtwo', 'pikachu', 'squirtle']

Y = np_utils.to_categorical(labels, num_classes)

Image shuffle, Train test(validation) split and data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
x, y = shuffle(img_data, Y, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=4)  # can change the test_size

# data augmentation
aug=ImageDataGenerator(rotation_range=25, 
                       width_shift_range=0.1, 
                       height_shift_range=0.1, shear_range=0.2, 
                       zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

input_shape = img_data[0].shape
print(input_shape) # make sure it is (128, 128, 1), sometimes not stable in google colab

Build a sequential CNN model; met problem: AttributeError: module 'tensorflow._api.v2.config' has no attribute 'experimental_list_devices'
Solution: 

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", input_shape=input_shape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1)) # was axis=1
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# softmax classifier
model.add(Dense(5))  # Last Dense input is used to specify the number of classes to predict
model.add(Activation("softmax"))

model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=["accuracy"])

Summary of the model

In [ ]:
model.summary()

show the training; it works once the model step above works

In [ ]:
# cite "aug" use data augmentation in model training
hist = model.fit(x=aug.flow(X_train, y_train, batch_size=32), epochs=40,
                 verbose=1, validation_data=(X_test, y_test))


Plot the accuracy and loss

In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(40) # range(number of epochs)

plt.figure(1, figsize=(7, 5))
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train', 'val'])
# print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.savefig('trial_psi_loss.png', bbox_inches='tight')

plt.figure(2, figsize=(7, 5))
plt.plot(xc, train_acc)
plt.plot(xc, val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train', 'val'], loc=4)
# print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.savefig('trial_psi_acc.png', bbox_inches='tight')

# evaluate score
score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])  # validation accuracy


Plot the Confusion Matrix for validation data(X_test, y_test)

In [ ]:
# y_test[1] need round up
y_testnew=np.argmax(y_test, axis=1)
y_testnew[1]


In [ ]:
# define the data to be ploted in confusion matrix(cm), here we use validation data
predicted_classes=model.predict_classes(X_test)
predicted_classes[1]

In [ ]:
# import libraries and define cm(confusion matrix)
from sklearn import metrics
from sklearn.metrics import confusion_matrix
cm = metrics.confusion_matrix(y_testnew, predicted_classes)


Define a function to plot the confusion matrix

In [ ]:
#Defining function for confusion matrix plot
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Computing confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

# Visualizing
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

   # Rotating the tick labels and setting their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Looping over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
np.set_printoptions(precision=2)


Specify an index of pokemon in displayed confusion matrix, i.e. class_names

In [ ]:
class_names= names
# as it was defined earlier in this program, names = ['bulbasaur', 'charmander', 'mewtwo', 'pikachu', 'squirtle']

Plot non-nomalized cm

In [ ]:
#Non-Normalized Confusion Matrix
plt.figure(figsize=(20,20))
plot_confusion_matrix(y_testnew, predicted_classes, classes = class_names, title='Non-Normalized Confusion matrix')
plt.show()


Plot normalized cm

In [ ]:
#Normalized Confusion Matrix
plt.figure(figsize=(35,35))
plot_confusion_matrix(y_testnew, predicted_classes, classes = class_names, normalize=True, title='Non-Normalized Confusion matrix')
plt.show()


save the model

In [ ]:
model.save('/content/gdrive/My Drive/Pokemonk/Poke_modelk.h5')  # save your model with h5py